## Credentials

In [ ]:
API_KEY = ""
location = 'us-south'
space_id = ""

In [ ]:
wml_credentials = {
    "apikey": API_KEY,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}


In [ ]:
from ibm_watson_machine_learning import APIClient
import requests
import json

client = APIClient(wml_credentials)

In [ ]:
DB2_HOST = ''
DB2_PORT = ''
DB2_USERNAME = ''
DB2_PASSWORD = ''
DB2_DB_NAME = ''
dsn_protocol = ""               # i.e. "TCPIP"
dsn_driver   = "" # Don't change

## Execute SQL

In [ ]:
sql_query = "SELECT DISTINCT STATE FROM CMY49613.COMMODITY"

In [ ]:
def connect_to_db2(db2_dbname, db2_host, db2_port, db2_username, db2_password):
    import ibm_db
    
    # Connect to DB2
    dsn = ("DRIVER={{IBM DB2 ODBC DRIVER}};" "DATABASE={0};" "HOSTNAME={1};" "PORT={2};" "PROTOCOL=TCPIP;" "UID={3};" "PWD={4};SECURITY=SSL").format(db2_dbname, db2_host, db2_port, db2_username, db2_password)
    options = { ibm_db.SQL_ATTR_AUTOCOMMIT:  ibm_db.SQL_AUTOCOMMIT_ON }
    conn = ibm_db.connect(dsn, "", "",options)

    if conn:
        print("Connected to the database.")
        return conn
    else:
        print("Failed to connect to the database.")
        return None
        
conn = connect_to_db2(DB2_DB_NAME, DB2_HOST, DB2_PORT, DB2_USERNAME, DB2_PASSWORD)

In [ ]:
def query_table(conn, sql_query):
    import ibm_db
    
    stmt = ibm_db.exec_immediate(conn, sql_query)

    # Fetch and print the results
    result = ibm_db.fetch_assoc(stmt)
    result_list = []
    while result:
        new_data = {}
        for key in result.keys():
            if key == "ARRIVAL_DATE":
                new_data[key] = result[key].strftime("%Y-%m-%d")
            else:
                new_data[key] = result[key]

        result_list.append(new_data)
        result = ibm_db.fetch_assoc(stmt)

    return result_list

result = query_table(conn, sql_query)
result

## Compile Functions

In [ ]:
payload = {
    "input_data": [{
        "fields": ["sql_query"],
        "values": [[sql_query]]
    }]
}

In [ ]:
def execute_SQL_query():
    
    def connect_to_db2(db2_dbname, db2_host, db2_port, db2_username, db2_password):
        import ibm_db
        
        # Connect to DB2
        dsn = ("DRIVER={{IBM DB2 ODBC DRIVER}};" "DATABASE={0};" "HOSTNAME={1};" "PORT={2};" "PROTOCOL=TCPIP;" "UID={3};" "PWD={4};SECURITY=SSL").format(db2_dbname, db2_host, db2_port, db2_username, db2_password)
        options = { ibm_db.SQL_ATTR_AUTOCOMMIT:  ibm_db.SQL_AUTOCOMMIT_ON }
        conn = ibm_db.connect(dsn, "", "",options)

        if conn:
            print("Connected to the database.")
            return conn
        else:
            print("Failed to connect to the database.")
            return None
        
    def query_table(conn, sql_query):
        import ibm_db
        
        stmt = ibm_db.exec_immediate(conn, sql_query)

        # Fetch and print the results
        result = ibm_db.fetch_assoc(stmt)
        result_list = []
        while result:
            new_data = {}
            for key in result.keys():
                if key == "ARRIVAL_DATE":
                    new_data[key] = result[key].strftime("%Y-%m-%d")
                else:
                    new_data[key] = result[key]

            result_list.append(new_data)
            result = ibm_db.fetch_assoc(stmt)

        return result_list
    
    def get_result(payload):
        DB2_HOST = ''
        DB2_PORT = ''
        DB2_USERNAME = ''
        DB2_PASSWORD = ''
        DB2_DB_NAME = ''
        dsn_protocol = ""               # i.e. "TCPIP"
        dsn_driver   = "" # Don't change

        sql_query = payload["input_data"][0]["values"][0][0]
        
        conn = connect_to_db2(DB2_DB_NAME, DB2_HOST, DB2_PORT, DB2_USERNAME, DB2_PASSWORD)
        result = query_table(conn, sql_query)
        
        return {
            "predictions": [{"fields": ["result"], "values": [result]}]
        }
        
    
    return get_result

In [ ]:
result = execute_SQL_query()(payload)
result

## Create deployment space

In [ ]:
client.set.default_space(space_id)
sofware_spec_uid = client.software_specifications.get_id_by_name("runtime-23.1-py3.10")
meta_data = {
    client.repository.FunctionMetaNames.NAME: 'execute-SQL-query',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}
function_details = client.repository.store_function(meta_props=meta_data, function=execute_SQL_query)

In [ ]:
function_uid = client.repository.get_function_uid(function_details)
# Deploy the stored function

metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "deployment-execute-SQL-query",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}
function_deployment_details = client.deployments.create(function_uid, meta_props=metadata)

## Test if deployment works

In [ ]:
# Get the endpoint URL of the function deployment just created
function_deployment_id = client.deployments.get_uid(function_deployment_details)
function_deployment_endpoint_url = client.deployments.get_scoring_href(function_deployment_details)
print("Function deployment id: {}".format(function_deployment_id))
print("Endpoint URL: {}".format(function_deployment_endpoint_url))

In [ ]:
result = client.deployments.score(function_deployment_id, payload)
if "error" in result:
    print(result["error"])
else:
    print(result)